In [1]:
# Import necessary module
from sqlalchemy import create_engine
import pandas as pd

# Create engine: engine
engine = create_engine('sqlite:///programming.sqlite')

In [2]:
# What are the tables in the database?

# Save the table names to a list: table_names
table_names  = engine.table_names()

# Print the table names to the shell
print(table_names)

# See the data model here: http://chinookdatabase.codeplex.com/wikipage?title=Chinook_Schema&referringTitle=Home


['Grade']


In [3]:
# The power of SQL lies in relationships between tables: INNER JOIN

# Execute query and store records in DataFrame: df
df = pd.read_sql_query("select * from Grade",engine)

# drop the unnecessary column
df = df.drop('index', 1)

# print the number of unique students
print(len(df['Aluno_ID'].unique()))

# Print head of DataFrame
df.head()


766


,Aluno_ID,Período,Disciplina,Situaçăo,Média Final,Unidade I,Unidade II,Unidade III
0,0,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,7.0,4.9,9.0,7.0
1,1,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,7.3,8.0,7.0,7.0
2,2,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,9.3,9.5,8.3,10.0
3,3,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,7.3,6.5,7.0,8.3
4,4,2014.1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,APROVADO,7.5,5.5,8.0,9.0


In [19]:
semesters = pd.read_sql_query("SELECT DISTINCT Período FROM Grade", engine)

In [21]:
semesters = semesters[semesters.columns[0]].tolist()
semesters

[2014.1, 2014.2, 2015.1, 2015.2, 2016.1, 2016.2]

In [30]:
studentsbySemester = []
for s in semesters:
    students = pd.read_sql_query(
        "SELECT Aluno_ID, Período, Disciplina, \"Situação\", \"Média Final\" FROM Grade WHERE Período = " + str(s), 
        engine)
    studentsbySemester.append(students)